In [87]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy.stats as stats
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
from jupyter_dash import JupyterDash


In [88]:

np.random.seed(42)

n_obligors = 100  # number of companies
T =  60 # time horizon in years
n_simulations = 50
sectors = ['Tech', 'Energy', 'Finance', 'Healthcare', 'Consumer']
sector_assignments = np.random.choice(sectors, size=n_obligors)
sector_rho = {'Tech': 0.3, 'Energy': 0.4, 'Finance': 0.5, 'Healthcare': 0.2, 'Consumer': 0.25}

def generate_correlation_matrix(sector_rho):
    matrix = np.eye(n_obligors)
    for i in range(n_obligors):
        for j in range(n_obligors):
            if i != j:
                if sector_assignments[i] == sector_assignments[j]:
                    matrix[i, j] = sector_rho[sector_assignments[i]]
                else:
                    matrix[i, j] = 0.1
    return matrix

def generate_recovery_rates(n, alpha, beta):
    return np.random.beta(alpha, beta, size=n)

sector_rho_initial = {'Tech': 0.3, 'Energy': 0.4, 'Finance': 0.5, 'Healthcare': 0.2, 'Consumer': 0.25}
corr_matrix = generate_correlation_matrix(sector_rho_initial)

L = np.linalg.cholesky(corr_matrix)

# Stochastic Recovery: Beta distribution parameters
recovery_alpha, recovery_beta = 2, 5

def generate_recovery_rates(n, alpha, beta):
    return np.random.beta(alpha, beta, size=n)

default_probs_base = np.full(n_obligors, 0.05)
default_probs = default_probs_base.copy()

# Tranche boundaries with base correlation smile
tranches = {
    'Equity': (0.0, 0.03, 0.25),
    'Mezzanine': (0.03, 0.07, 0.30),
    'Senior': (0.07, 0.10, 0.35)
}


In [89]:

def simulate_losses(macro_shock_mean, macro_shock_std, sector_corr_scale, recov_alpha, recov_beta, stress_type='normal'):
    sector_rho = {'Tech': 0.3 * sector_corr_scale, 'Energy': 0.4 * sector_corr_scale, 'Finance': 0.5 * sector_corr_scale, 'Healthcare': 0.2 * sector_corr_scale, 'Consumer': 0.25 * sector_corr_scale}
    corr_matrix = generate_correlation_matrix(sector_rho)
    L = np.linalg.cholesky(corr_matrix)

    losses = []
    yearly_losses = []

    pre_Z = np.random.randn(n_simulations, n_obligors, T)

    for sim in range(n_simulations):
        cumulative_loss = 0
        yearly_cum_loss = []
        current_default_probs = default_probs_base.copy()

        for year in range(T):
            Z = pre_Z[sim, :, year]
            correlated_Z = L @ Z
            U = stats.norm.cdf(correlated_Z)

            if stress_type == 'recession':
                current_default_probs = np.minimum(current_default_probs * 1.5, 1.0)
            elif stress_type == 'credit_crunch':
                sector_rho = {k: min(v + 0.2, 0.95) for k, v in sector_rho.items()}
                corr_matrix = generate_correlation_matrix(sector_rho)
                L = np.linalg.cholesky(corr_matrix)
            elif stress_type == 'market_shock':
                current_default_probs = np.minimum(current_default_probs + 0.1, 1.0)

            defaults = (U < current_default_probs / T).astype(int)
            recovery_rates = generate_recovery_rates(n_obligors, recov_alpha, recov_beta)
            loss = np.sum(defaults * (1 - recovery_rates)) / n_obligors
            cumulative_loss += loss
            yearly_cum_loss.append(cumulative_loss)

            macro_shock = np.random.normal(macro_shock_mean, macro_shock_std)
            current_default_probs = np.minimum(current_default_probs * (1 + macro_shock), 1.0)

        losses.append(cumulative_loss)
        yearly_losses.append(yearly_cum_loss)

    return np.array(losses), np.array(yearly_losses)

In [90]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Dynamic CDO Stress Testing Dashboard"),
    html.Div([
        html.Label("Macro Shock Mean:"),
        dcc.Slider(id='macro_shock_mean', min=0.0, max=0.05, step=0.01, value=0.01),

        html.Label("Macro Shock Std Dev:"),
        dcc.Slider(id='macro_shock_std', min=0.0, max=0.02, step=0.005, value=0.005),

        html.Label("Sector Correlation Scaling:"),
        dcc.Slider(id='sector_corr_scale', min=0.5, max=2.0, step=0.1, value=1.0),

        html.Label("Recovery Rate Skewness (Alpha):"),
        dcc.Slider(id='recov_alpha', min=0.5, max=5.0, step=0.5, value=2.0),

        html.Label("Recovery Rate Skewness (Beta):"),
        dcc.Slider(id='recov_beta', min=0.5, max=5.0, step=0.5, value=5.0),

        html.Label("Stress Scenario:"),
        dcc.Dropdown(
            id='stress_type',
            options=[
                {'label': 'Normal', 'value': 'normal'},
                {'label': 'Recession', 'value': 'recession'},
                {'label': 'Credit Crunch', 'value': 'credit_crunch'},
                {'label': 'Market Shock', 'value': 'market_shock'}
            ],
            value='normal'
        ),
    ], style={'columnCount': 2}),

    dcc.Graph(id='loss_animation')
])

@app.callback(
    Output('loss_animation', 'figure'),
    Input('macro_shock_mean', 'value'),
    Input('macro_shock_std', 'value'),
    Input('sector_corr_scale', 'value'),
    Input('recov_alpha', 'value'),
    Input('recov_beta', 'value'),
    Input('stress_type', 'value')
)
def update_graph(macro_shock_mean, macro_shock_std, sector_corr_scale, recov_alpha, recov_beta, stress_type):
    _, yearly_losses = simulate_losses(macro_shock_mean, macro_shock_std, sector_corr_scale, recov_alpha, recov_beta, stress_type)

    frames = [go.Frame(data=[go.Histogram(x=yearly_losses[:, year], nbinsx=50)], name=str(year)) for year in range(T)]

    fig = go.Figure(
        data=[go.Histogram(x=yearly_losses[:, 0], nbinsx=50)],
        layout=go.Layout(
            title="Portfolio Loss Distribution Over Time",
            xaxis_title="Cumulative Loss",
            yaxis_title="Density",
            updatemenus=[{
                'buttons': [
                    {
                        'args': [None, {'frame': {'duration': 60, 'redraw': True}, 'fromcurrent': True}],
                        'label': 'Play',
                        'method': 'animate'
                    }
                ],
                'type': 'buttons'
            }],
            transition={'duration': 20, 'easing': 'cubic-in-out'}
        ),
        frames=frames
    )

    return fig

if __name__ == '__main__':
    app.run(debug=True)
